In [1]:
import pandas as pd
import re
import country_converter as coco
import csv

# SETUP & DATA ACQUISITION

# Loading the UNCTAD ISDS Navigator data
# We use header=11 to skip the metadata rows in the standard UNCTAD Excel export
path = "UNCTAD-ISDS-Navigator-data-set-31December2023.xlsx"
raw_df = pd.read_excel(path, header=11)

# Feature selection and renaming for easier handling
df = raw_df[[
    "RESPONDENT STATE",
    "HOME STATE OF INVESTOR",
    "ECONOMIC SECTOR",
    "SUMMARY OF THE DISPUTE",
    "AMOUNT CLAIMED (EXPRESSED IN MILLIONS)",
    "YEAR OF INITIATION"
]].rename(columns={
    "RESPONDENT STATE": "respondent_state",
    "HOME STATE OF INVESTOR": "investor_home_state",
    "ECONOMIC SECTOR": "economic_sector",
    "SUMMARY OF THE DISPUTE": "summary_of_dispute",
    "AMOUNT CLAIMED (EXPRESSED IN MILLIONS)": "amount_claimed_raw",
    "YEAR OF INITIATION": "year"
})


# 1. FEATURE EXTRACTION & CLEANING

def parse_claim_amount(x):
    """
    Extracts numerical USD values from messy UNCTAD strings.
    Handles commas and prefers parenthetical USD amounts if present.
    """
    if pd.isna(x): return None
    s = str(x).replace(",", "")
    
    # Priority 1: Match amount inside parentheses (e.g. "123.45 USD")
    m = re.search(r"\(\s*([0-9]+(?:\.[0-9]+)?)\s*USD\s*\)", s)
    if m: return float(m.group(1))
    
    # Priority 2: Standard USD suffix
    m = re.search(r"([0-9]+(?:\.[0-9]+)?)\s*USD\b", s)
    if m: return float(m.group(1))
    
    return None

df["amount_claimed_musd"] = df["amount_claimed_raw"].apply(parse_claim_amount)
df["year"] = pd.to_numeric(df["year"], errors="coerce")


# 2. SECTOR DYNAMICS (BINARY MATRIX)

# ISDS disputes often involve multiple sectors per case.
# We create a binary design matrix to allow for granular analysis of sector-specific risks.
sector_series = (
    df["economic_sector"]
    .fillna("")
    .astype(str)
    .str.split(r";\s*\n")
    .explode()
    .str.strip()
)

# Remove non-informative entries
sector_series = sector_series[~sector_series.isin(["", "Data not available"])]

# Generate dummy variables and collapse them back to the original row level
sector_design = pd.get_dummies(sector_series, prefix="sector").groupby(level=0).max()
df = df.join(sector_design)


# 3. GEOSPATIAL HARMONIZATION (ISO3 CODES)

cc = coco.CountryConverter()
ISO_CACHE = {}

def get_iso3_codes(country_string):
    """
    Standardizes state names to ISO3 codes. Includes a cache to 
    significantly speed up processing for large datasets.
    """
    if pd.isna(country_string): return None
    
    s = str(country_string).strip()
    if s.lower() in ["data not available", "selection deleted", "european union", ""]:
        return None
    
    # Some disputes list multiple home states
    countries = [c.strip() for c in s.split(";\n") if c.strip()]
    codes = []
    
    for c in countries:
        if c not in ISO_CACHE:
            res = cc.convert(names=c, to="ISO3", not_found=None)
            ISO_CACHE[c] = res
        if ISO_CACHE[c] and ISO_CACHE[c] != "not found":
            codes.append(ISO_CACHE[c])
            
    return "; ".join(sorted(set(codes))) if codes else None

# Applying harmonization to both the respondent and investor countries
for col in ["respondent_state", "investor_home_state"]:
    df[f"{col}_iso3"] = df[col].apply(get_iso3_codes)


# DOWNLOADING FILE

# Save the final analytical dataset
df.to_csv("ISDS_processed_dataset1.csv", index=False)

print(f"Pipeline complete. Dataset dimensions: {df.shape}")
print("Final file saved as: ISDS_processed_dataset1.csv")

Pipeline complete. Dataset dimensions: (1332, 27)
Final file saved as: ISDS_processed_dataset1.csv
